# Modeling 
## Early Stage Diabetes Risk Prediction

### Objective
In this notebook, we will train multiple machine learning models to predict whether a patient is at risk of diabetes. We will compare their performance to select the best one for our final application.

###  Models to Train
1.  **Logistic Regression:** A simple, interpretable baseline.
2.  **Random Forest:** A powerful ensemble method that handles non-linear relationships well.
3.  **Support Vector Machine (SVM):** Excellent for finding clear margins between classes.
4.  **K-Nearest Neighbors (KNN):** A simple distance-based algorithm (relies heavily on our scaling step).

###  Evaluation Metrics
Since this is a medical "Risk Prediction" problem, **Recall** (Sensitivity) is our most critical metric.
* **Recall:** "Out of all the people who actually have diabetes, how many did we catch?" (We do NOT want to miss a sick patient).
* **Accuracy:** Overall correctness.
* **F1-Score:** A balance between Precision and Recall.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Set style
sns.set_theme(style="whitegrid")

# 1. Load the Processed Data
X_train = pd.read_csv('../data/processed/X_train.csv')
X_val = pd.read_csv('../data/processed/X_val.csv')
X_test = pd.read_csv('../data/processed/X_test.csv')

y_train = pd.read_csv('../data/processed/y_train.csv').values.ravel() # .values.ravel() converts DF to 1D array
y_val = pd.read_csv('../data/processed/y_val.csv').values.ravel()
y_test = pd.read_csv('../data/processed/y_test.csv').values.ravel()

print("Data Loaded Successfully!")
print(f"Training Features: {X_train.shape}")

Data Loaded Successfully!
Training Features: (150, 15)


## 1. Model Training & Validation Loop
Instead of writing code 4 times, we will create a dictionary of models and loop through them. This ensures every model is trained and evaluated in exactly the same way.


In [2]:
# Initialize the models
models = {
    "Logistic Regression": LogisticRegression(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel='linear', probability=True, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

# Dictionary to store results
results = []

print(" Starting Training Loop...\n")

for name, model in models.items():
    print(f"🔹 Training {name}...")
    
    # 1. Train the model
    model.fit(X_train, y_train)
    
    # --- Check Overfitting (NEW STEP) ---
    # Predict on Training Data
    y_train_pred = model.predict(X_train)
    train_acc = accuracy_score(y_train, y_train_pred)
    
    # 2. Predict on Validation Set
    y_val_pred = model.predict(X_val)
    
    # 3. Calculate Validation Metrics
    val_acc = accuracy_score(y_val, y_val_pred)
    prec = precision_score(y_val, y_val_pred)
    rec = recall_score(y_val, y_val_pred)
    f1 = f1_score(y_val, y_val_pred)
    
    # Print comparison immediately
    print(f"   -> Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")
    
    # 4. Append to results
    results.append({
        "Model": name,
        "Train Accuracy": train_acc, # Added this to the final table too
        "Val Accuracy": val_acc,
        "Precision": prec,
        "Recall": rec,
        "F1-Score": f1
    })

print("\n Training Complete!")

 Starting Training Loop...

🔹 Training Logistic Regression...
   -> Train Acc: 0.9000 | Val Acc: 0.8400
🔹 Training Random Forest...
   -> Train Acc: 1.0000 | Val Acc: 0.9200
🔹 Training SVM...
   -> Train Acc: 0.9067 | Val Acc: 0.7600
🔹 Training KNN...
   -> Train Acc: 0.9333 | Val Acc: 0.7200

 Training Complete!


## 2. Model Comparison
Let's convert our results into a DataFrame and visualize them to pick the winner.

In [8]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Sort by Recall (since it's a medical problem, Recall is king)
results_df = results_df.sort_values(by='Recall', ascending=False)

# Display the table
print("--- Model Performance on Validation Set ---")
display(results_df.style.background_gradient(cmap='Greens'))



--- Model Performance on Validation Set ---


,Model,Train Accuracy,Val Accuracy,Precision,Recall,F1-Score
1,Random Forest,1.000000,0.920000,0.942857,0.942857,0.942857
0,Logistic Regression,0.900000,0.840000,1.000000,0.771429,0.870968
2,SVM,0.906667,0.760000,0.896552,0.742857,0.812500
3,KNN,0.933333,0.720000,0.956522,0.628571,0.758621


###  Model Selection & Analysis

Based on the training and validation results above, we have selected **Random Forest** as our Champion Model.

#### 1. Performance Breakdown
* **Random Forest** achieved the highest **Validation Accuracy (92%)**, outperforming all other models.
* **Logistic Regression** came in second **(84%)**, suggesting the data is relatively clean, but the Random Forest captured the complex non-linear patterns better.

#### 2. Analyzing the "Perfect" Training Score (1.0000)
We notice that the Random Forest has a **Training Accuracy of 100%**.
* *Is this overfitting?* In many models, yes. However, for Random Forests, this is expected behavior. Decision Trees are designed to split data until leaves are pure.
* *The Verdict:* Since the **Validation Accuracy (92%)** is also very high, the gap is small (8%). This confirms the model is **generalizing well** and not just "memorizing" the data.

####  Final Decision
We choose **Random Forest** as the final model for evaluation on the Test Set and for the deployment application.

## 3. Hyperparameter Tuning Results
We utilized `GridSearchCV` to exhaustively search for the optimal model architecture. By testing combinations of tree depth, leaf size, and splitting criteria, we improved the model's ability to generalize.

**Key Optimization:**
* **Metric:** We optimized for **Recall** to ensure the model prioritizes catching positive diabetes cases over pure accuracy.
* **Best Parameters:** The search identified that a Random Forest with the settings above provides the most stable performance on unseen data.

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# 1. Define the "Grid" of settings to test
# We try different numbers of trees, depths, and splitting rules
param_grid = {
    'n_estimators': [50, 100, 200],      # How many trees?
    'max_depth': [None, 10, 20, 30],     # How deep can they grow? (None = unlimited)
    'min_samples_split': [2, 5, 10],     # Minimum samples to create a split
    'min_samples_leaf': [1, 2, 4],       # Minimum samples in a final leaf
    'criterion': ['gini', 'entropy']     # The math formula for splitting
}

# 2. Initialize the Base Model
rf = RandomForestClassifier(random_state=42)

# 3. Setup GridSearchCV
# cv=5 means it validates 5 times for EVERY combination
grid_search = GridSearchCV(estimator=rf, 
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='recall',  # OPTIMIZE FOR RECALL
                           n_jobs=-1,         # Use all CPU cores
                           verbose=2)         # Show progress text

# 4. Run the Search (Training)
print("⚙️ Tuning Random Forest... This might take a minute.")
grid_search.fit(X_train, y_train)

# 5. Get the Best Results
best_rf = grid_search.best_estimator_

print("\n✅ BEST PARAMETERS FOUND:")
print(grid_search.best_params_)
print(f"\n🏆 Best Cross-Validation Recall: {grid_search.best_score_:.4f}")

⚙️ Tuning Random Forest... This might take a minute.
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.6s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.1s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_l

##  Hyperparameter Tuning Results & Analysis

We performed a `GridSearchCV` to optimize the Random Forest model for **Recall**, aiming to minimize missed diabetes cases (False Negatives).

### 1. The Outcome
* **Baseline Recall (Single Validation Split):** `94.28%`
* **Tuned Cross-Validation Recall (Average of 5 Folds):** `94.14%`
* **Best Parameters:** `{n_estimators: 100, max_depth: None, criterion: 'gini'}`

### 2. Interpretation: "Stability over Complexity"
The Grid Search analysis reveals two critical insights:

1.  **Validation of Robustness:** The Cross-Validation score (`94.14%`) is statistically identical to our initial single-split score (`94.28%`).
    * *Significance:* This proves our model is **stable**. The initial high performance was not a result of a "lucky" train-test split but a genuine ability to detect the disease patterns consistently.

2.  **Optimal Complexity:** The search converged on the standard parameter set (e.g., `n_estimators=100`, `max_depth=None`).
    * *Significance:* The dataset contains strong, clear signals (like Polyuria and Polydipsia). The model does not require heavy regularization (limiting tree depth) or boosting to achieve high performance.

### 3. Conclusion
We will proceed with the **Best Estimator** identified by the Grid Search. While the numerical score did not increase, the **confidence** in the model's reliability has improved significantly because this result is backed by 5-fold cross-validation rather than a single test.

## Save the best model (Random Forest)

In [10]:
import joblib
import os

# 1. Select the Tuned Model
# We use 'best_rf' which comes from the GridSearchCV we just ran
# (Ensure you ran the Grid Search cell above this one!)
final_model_to_save = best_rf 

# 2. Define the path
# We overwrite the old file so the App uses the new tuned version automatically
save_path = '../models/best_model.joblib'

# 3. Create the directory if it doesn't exist (safety check)
os.makedirs('../models', exist_ok=True)

# 4. Save the model
joblib.dump(final_model_to_save, save_path)

print(f" Success! The TUNED model has been saved to: {save_path}")

# 5. Verify it exists
print("Current files in 'models' folder:")
print(os.listdir('../models'))

 Success! The TUNED model has been saved to: ../models/best_model.joblib
Current files in 'models' folder:
['target_encoder.joblib', 'best_model.joblib', 'preprocessor.joblib']


### Cross-Validation Check (Reliability Test)
We perform **Cross-Validation** to ensure our model's performance is stable. We specifically monitor **Recall** to confirm we are consistently catching positive cases across different subsets of data.

In [11]:
from sklearn.model_selection import cross_val_score

# Example for Random Forest
rf_model = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='recall')

print(f"Cross-Validation Recall Scores: {scores}")
print(f"Average Recall: {scores.mean():.4f}")

Cross-Validation Recall Scores: [0.80952381 1.         1.         0.9        0.95      ]
Average Recall: 0.9319
